In [1]:
%pip install python-telegram-bot ollama nest_asyncio dotenv

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import logging
import os
from dotenv import load_dotenv
from telegram import Update
from telegram.ext import ApplicationBuilder, CommandHandler, MessageHandler, filters
import ollama
import nest_asyncio

nest_asyncio.apply()

# Включаем ведение журнала
logging.basicConfig(format='%(asctime)s - %(name)s - %(levelname)s - %(message)s', level=logging.INFO)

# Токен, который вы получили от @BotFather
load_dotenv()
TOKEN = os.environ.get('BOT_TOKEN')

# Словарь для хранения данных пользователей
user_ids = {}
context_memory = {}

# Функция для обработки команды /start
async def start(update: Update, context) -> None:
    await update.message.reply_text('Привет! Я чат-бот. Чем могу помочь?')

# Функция для обработки обычных сообщений
async def handle_message(update: Update, context):
    user_id = update.effective_user.id
    if user_id not in user_ids:
        user_ids[user_id] = {'last_message': None, 'preferences': {}}
        context_memory[user_id] = []

    message_text = update.message.text
    context_messages = context_memory[user_id]

    # Добавляем новое сообщение в контекст
    context_messages.append({'role': 'user', 'content': message_text})

    # Ограничиваем историю контекста последними 8 сообщениями
    context_memory[user_id] = context_messages[-8:]

    try:
        # Call the ollama.chat function with the context messages
        response = ollama.chat(model='deepseek-r1:8b', messages=context_memory[user_id])
        # Отправляем ответ пользователю
        await update.message.reply_text(response['message']['content'].split("</think>")[-1].strip())
    except Exception as e:
        logging.error(f"Error while getting response from ollama: {e}")
        await update.message.reply_text('Произошла ошибка, попробуйте позже.')

# Основная функция
async def main() -> None:
    application = ApplicationBuilder().token(TOKEN).build()

    application.add_handler(CommandHandler('start', start))
    application.add_handler(MessageHandler(filters.TEXT & ~filters.COMMAND, handle_message))

    await application.run_polling()

if __name__ == '__main__':
    import asyncio
    asyncio.run(main())

2025-03-25 22:57:12,339 - httpx - INFO - HTTP Request: POST https://api.telegram.org/bot7696980218:AAH9hdrW5tuhJqHEedo6xqa8mefddKaG7AE/getMe "HTTP/1.1 200 OK"
2025-03-25 22:57:12,402 - httpx - INFO - HTTP Request: POST https://api.telegram.org/bot7696980218:AAH9hdrW5tuhJqHEedo6xqa8mefddKaG7AE/deleteWebhook "HTTP/1.1 200 OK"
2025-03-25 22:57:12,403 - telegram.ext.Application - INFO - Application started
2025-03-25 22:57:18,447 - httpx - INFO - HTTP Request: POST https://api.telegram.org/bot7696980218:AAH9hdrW5tuhJqHEedo6xqa8mefddKaG7AE/getUpdates "HTTP/1.1 200 OK"
2025-03-25 22:57:48,214 - httpx - INFO - HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
2025-03-25 22:57:48,587 - httpx - INFO - HTTP Request: POST https://api.telegram.org/bot7696980218:AAH9hdrW5tuhJqHEedo6xqa8mefddKaG7AE/sendMessage "HTTP/1.1 200 OK"
2025-03-25 22:57:58,407 - httpx - INFO - HTTP Request: POST https://api.telegram.org/bot7696980218:AAH9hdrW5tuhJqHEedo6xqa8mefddKaG7AE/getUpdates "HTTP/1.1